In [1]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import Layout, AppLayout

import techminer.analytics as tc
import techminer.plots as plt
from techminer.analytics import load_scopus

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)

df = load_scopus(df)

2020-06-01 03:25:37,587 - INFO - Renaming and selecting columns ...
2020-06-01 03:25:37,596 - INFO - Formatting author names ...
2020-06-01 03:25:43,139 - INFO - Removing part of titles in foreing languages ...
2020-06-01 03:25:43,143 - INFO - Fusioning author and index keywords ...
2020-06-01 03:25:43,157 - INFO - NumExpr defaulting to 8 threads.
2020-06-01 03:25:43,174 - INFO - Extracting countries from affiliations ...
2020-06-01 03:25:49,155 - INFO - Extracting institutions from affiliations ...
2020-06-01 03:25:49,165 - INFO - Extracting country of 1st author ...
2020-06-01 03:25:49,168 - INFO - Extracting affiliation of 1st author ...
2020-06-01 03:25:49,173 - INFO - Counting number of authors ...


In [2]:
COLORMAPS = [
    "Greys",
    "Purples",
    "Blues",
    "Greens",
    "Oranges",
    "Reds",
    "YlOrBr",
    "YlOrRd",
    "OrRd",
    "PuRd",
    "RdPu",
    "BuPu",
    "GnBu",
    "PuBu",
    "YlGnBu",
    "PuBuGn",
    "BuGn",
    "YlGn",
    "Pastel1",
    "Pastel2",
    "Paired",
    "Accent",
    "Dark2",
    "Set1",
    "Set2",
    "Set3",
    "tab10",
    "tab20",
    "tab20b",
    "tab20c",
]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st" "Institutions",
    "Keywords",
    "Source title",
]

In [6]:
def co_occurrence_analysis(x):
    def compute_by_term(
        rows, columns, analysis_type, row_order, column_order, cmap, minmax
    ):
        #
        if analysis_type == "Frequency":
            matrix, limit_value = tc.co_occurrence(
                x,
                rows,
                columns,
                as_matrix=True,
                minmax=(selection_range.value[0], selection_range.value[1]),
                keywords=None,
                retmaxval=True,
            )
        else:
            matrix, limit_value = tc.co_citation(
                x,
                rows,
                columns,
                as_matrix=True,
                minmax=(selection_range.value[0], selection_range.value[1]),
                keywords=None,
                retmaxval=True,
            )
        #
        values = selection_range.value        
        if values[1] > limit_value or selection_range.max < limit_value:
            selection_range.max = limit_value
            if values[0] > limit_value:
                selection_range.min = 0
            #selection_range.value = [0, limit_value]
        #
        output.clear_output()
        with output:
            if len(matrix.columns) < 21 and len(matrix.index) < 21:
                display(matrix.style.background_gradient(cmap=cmap))
            else:
                display(matrix)  # .style.background_gradient(cmap=cmap)

    #
    PANEL_HEIGHT = "570px"
    #
    rows = widgets.Select(
        options=[z for z in COLUMNS if z in x.columns],
        ensure_option=True,
        disabled=False,
    )
    columns = widgets.Select(
        options=[z for z in COLUMNS if z in x.columns],
        ensure_option=True,
        disabled=False,
    )
    analysis_type = widgets.Dropdown(
        options=["Frequency", "Citation"], value="Frequency", disable=False,
    )
    selection_range = widgets.IntRangeSlider(
        value=[0, 1000],
        min=0,
        max=1000,
        step=1,
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
    )
    row_order = widgets.Dropdown(
        options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
        value="F/C desc.",
        disable=False,
    )
    column_order = widgets.Dropdown(
        options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
        value="F/C desc.",
        disable=False,
    )
    cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
    #
    output = widgets.Output()
    with output:
        display(
            widgets.interactive_output(
                compute_by_term,
                {
                    "rows": rows,
                    "columns": columns,
                    "analysis_type": analysis_type,
                    "row_order": row_order,
                    "column_order": column_order,
                    "cmap": cmap,
                    "minmax": selection_range,
                },
            )
        )
    #
    left_box = widgets.VBox(
        [
            widgets.VBox([widgets.Label(value="Rows:"), rows]),
            widgets.VBox([widgets.Label(value="Columns:"), columns]),
            widgets.VBox([widgets.Label(value="Analysis type:"), analysis_type]),
            widgets.VBox([widgets.Label(value="Range:"), selection_range]),
            widgets.VBox([widgets.Label(value="Row order:"), row_order]),
            widgets.VBox([widgets.Label(value="Column order:"), column_order]),
            widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
        ],
        layout=Layout(height=PANEL_HEIGHT, border="1px solid gray"),
    )
    right_box = widgets.VBox([output])

    return AppLayout(
        header=widgets.HTML(value=html_title("Co-occurrence analysis")),
        left_sidebar=left_box,
        center=right_box,
        right_sidebar=None,
        pane_widths=[2, 5, 0],
        pane_heights=["85px", 5, 0],
    )


##
## Debug
##
FIGSIZE = (12, 6)
PANEL_HEIGHT = "400px"


def html_title(x):
    return (
        "<h1>{}</h1>".format(x)
        + "<hr style='height:2px;border-width:0;color:gray;background-color:gray'>"
    )


co_occurrence_analysis(df)

AppLayout(children=(HTML(value="<h1>Co-occurrence analysis</h1><hr style='height:2px;border-width:0;color:gray…

In [ ]:
caption = widgets.Label(value='Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                              widgets.IntSlider(description='Target range 1')
dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))
display(caption, source_range, target_range1)

In [ ]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

In [ ]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

In [ ]:
def menu_descriptive_stats(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_analysis_by_term_per_term_per_year(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_occurrence(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_co_occurrence(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_autocorrelation(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_correlation(x):
    return widgets.Text("TO-DO")

In [ ]:
def menu_factor_analysis(x):
    return widgets.Text("TO-DO")

In [ ]:
def dashboard(x):
    m = [
        (menu_descriptive_stats, "Descriptive statistics"),
        (menu_analysis_by_year, "Analysis by year"),
        (menu_analysis_by_term, "Analysis by term"),
        (menu_analysis_by_term_per_year, "Analysis by term per year"),
        (menu_occurrence, "Occurrence analysis"),
        (menu_co_occurrence, "Co-occurrence analysis"),
        (menu_autocorrelation, "Autocorrelation analysis"),
        (menu_correlation, "Correlation analysis"),
        (menu_factor_analysis, "Factor analysis"),
    ]
    accordion = widgets.Accordion()
    widget_list = []
    for f, _ in m:
        widget_list.append(f(x))
    accordion.children = widget_list
    for index, t in enumerate(m):
        accordion.set_title(index, t[1])
    return accordion

In [ ]:
dashboard(df)

In [ ]:
df.columns